In [ ]:
import math
import os
import scipy
from scipy.optimize import lsq_linear
import numpy as np
from scipy.linalg import toeplitz
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, halfnorm
import random
from scipy.io import loadmat
import random
import pickle
import sys
from sklearn.linear_model import RidgeCV
sys.path.append(r"c:\Users\katie\OneDrive\Documents\GitHub\trial")
import PCA_Regress as pcar

In [2]:
base_path =r"c:\Users\katie\OneDrive\Desktop\Thesis"
with open(base_path+'\J_neu.pkl', "rb") as input_file:
    J_pickle = pickle.load(input_file)
del input_file

file_path = os.path.join(base_path, 'N_neu.pkl')
with open(file_path, "rb") as input_file:
    N_pickle = pickle.load(input_file)
del input_file

base_path =r"c:\Users\katie\OneDrive\Desktop\Thesis"
with open(base_path+'\J_mus.pkl', "rb") as input_file:
    J_pickle_m = pickle.load(input_file)
del input_file

ile_path = os.path.join(base_path, 'N_mus.pkl')
with open(ile_path, "rb") as input_file:
    N_pickle_m = pickle.load(input_file)
del input_file

# base_path = "/Users/kb6113/Desktop/Thesis"
# with open(base_path+'/J_neu.pkl', "rb") as input_file:
#     J_pickle = pickle.load(input_file)
# del input_file

# with open(base_path+'/J_mus.pkl', "rb") as input_file:
#     J_pickle_m = pickle.load(input_file)
# del input_file

J_all_tensor = J_pickle['J_all']['interpPSTH']
J_M1_tensor = J_pickle['J_M1']['interpPSTH']
J_PMd_tensor = J_pickle['J_PMd']['interpPSTH']
J_mus_tensor = J_pickle_m['interpPSTH']
J_mus_stack = np.vstack((J_mus_tensor, J_mus_tensor, J_mus_tensor, J_mus_tensor))


N_all_tensor = N_pickle['N_all']['interpPSTH']
N_M1_tensor = N_pickle['N_M1']['interpPSTH']
N_PMd_tensor = N_pickle['N_PMd']['interpPSTH']
N_mus_tensor = N_pickle_m['interpPSTH']

<>:2: SyntaxWarning: invalid escape sequence '\J'
<>:12: SyntaxWarning: invalid escape sequence '\J'
<>:2: SyntaxWarning: invalid escape sequence '\J'
<>:12: SyntaxWarning: invalid escape sequence '\J'
C:\Users\katie\AppData\Local\Temp\ipykernel_2664\3559308067.py:2: SyntaxWarning: invalid escape sequence '\J'
  with open(base_path+'\J_neu.pkl', "rb") as input_file:
C:\Users\katie\AppData\Local\Temp\ipykernel_2664\3559308067.py:12: SyntaxWarning: invalid escape sequence '\J'
  with open(base_path+'\J_mus.pkl', "rb") as input_file:


In [ ]:
conds = 108 
# 1. Create a range of all possible indexes
all_idx = np.arange(conds)

# 2. Shuffle the indexes to ensure random distribution across the splits
# This prevents any inherent order in the data from biasing the splits.
np.random.shuffle(all_idx)

# 3. Calculate the sizes of each partition
size_10_percent = int(np.round(conds * 0.1))
size_20_percent = int(np.round(conds * 0.2))
# The remainder goes to the 70% to ensure all indexes are used
size_70_percent = conds - size_10_percent - size_20_percent

# 4. Use numpy.split to divide the shuffled array
# The indices array specifies the points where the array is split
split_points = [size_10_percent, size_10_percent + size_20_percent]
split_sets = np.split(all_idx, split_points)

# 5. Assign the resulting arrays to their respective variables
set_10_percent = split_sets[0]
set_20_percent = split_sets[1]
set_70_percent = split_sets[2]


(11,)
(22,)
(75,)


In [ ]:
def best_lam(mus_rank, neu_rank, time_bins):
    """
    This function takes in the training data and will compute the best lambda value for ridge regression using cross-validation. It will return the best lambda
    value and the mean squared error for that lambda.

    Parameters: 
        mus_rank: a 2D numpy array of shape [ct, rank] which is the projection onto the first rank PCs
        neu_rank: a 2D numpy array of shape [ct, rank] which is the projection onto the first rank PCs
        M: the original muscle data of shape [conditions x time bins, muscles]

    Returns: 
        best_lambda: the best lambda value found during cross-validation
        mse: the mean squared error for the best lambda
    """

    # conditions in the sample 
    conds = int(neu_rank.shape[0] / time_bins)

    # Define a range of lambda values to test
    lambdas = np.logspace(-2, 3, 20)
    
    # Initialize variables to store the best lambda and its corresponding MSE
    best_lambda = None
    min_rmse = float('inf')

    # listing and shuffling all possible indexes 
    all_idx = np.arange(conds)
    np.random.shuffle(all_idx)

    # calculate the sizes of each set 
    size_10_percent = int(np.round(conds * 0.1))    # 10% for lambda 
    size_20_percent = int(np.round(conds * 0.2))    # 20% for testing

    # splitting the shuffled index array 
    split_points = [size_10_percent, size_10_percent + size_20_percent]
    split_sets = np.split(all_idx, split_points)

    # indexes of the split data 
    lam_idx = split_sets[0]           # 10% of data
    test_idx = split_sets[1]          # 20% of data 
    train_idx = split_sets[2]         # 70% of data


"""
    #split into testing and training, with a 20/80 split
    test_size = int(np.round(0.2 * conds))
    test_idx = np.random.choice(np.arange(conds), size = test_size, replace = False)
    
    # shaping back into a tensor 
    neu_tensor = pcar.shape_tensor(neu_rank, conds)
    mus_tensor = pcar.shape_tensor(mus_rank, conds)

    # isolating the test data 
    neu_test_tens = neu_tensor[test_idx, :, :]
    mus_test_tens = mus_tensor[test_idx, :, :]

    # indexes for training data 
    mask = np.ones(conds, dtype = bool)
    mask[test_idx] = False
    neu_training_tens = neu_tensor[mask, :, :]
    mus_training_tens= mus_tensor[mask, :, :]

    # shaping back into matrix for regression
    neu_test = pcar.shape_matrix(neu_test_tens)
    mus_test = pcar.shape_matrix(mus_test_tens)
    neu_train = pcar.shape_matrix(neu_training_tens)
    mus_train = pcar.shape_matrix(mus_training_tens)

    mse_vals = []
    rmse_vals = []

    # Perform cross-validation
    for lam in lambdas:
        # Fit a ridge regression model with the current lambda
        W_hat= pcar.regress(mus_train, neu_train, lam)[0]

        # Predict on the test sample
        check = neu_test @ W_hat
        
        mse = pcar.mse_fun(mus_test, check)
        mse_vals.append(mse)

        rmse = np.sqrt(mse)
        rmse_vals.append(rmse)

        # Update best lambda if current MSE is lower than previous minimu
        
        if rmse < min_rmse and lam != None:
            min_rmse = rmse
            best_lambda = lam
    print(">>> best_lam returning:", best_lambda)
    # Return the best lambda and its corresponding MSE         
    return best_lambda, min_rmse
    """